### Importanto as bibliotecas

In [1]:
import re #biblioteca para realizar buscas dentro do arquivo texto
import pickle #biblioteca para ler os arquivos

### Definindo uma classe automato e criando uma função que leia o automato dado um arquivo .txt

In [8]:
class automato:        
    def __init__(self, arqtxt): #recebe como parametro um arquivo texto contendo o automato
        
        #Retorna os simbolos do alfabeto, os estados do automato, o estado inicial e os estados finais
        quadrupla = re.search("\(.*?\)",dados) #Procura a parte da string entre parenteses e a retorna (com os parenteses) dentro de um objeto match
        quadrupla = quadrupla.group(0) #Extrai a string do objeto match
        quadrupla = quadrupla[1:len(quadrupla)-1] #Retira os parenteses do começo e do final

        #filtra os estados
        simbolos = re.search("\{.*?\}", quadrupla)
        simbolos = simbolos.group(0)
        simbolos = simbolos[1:len(simbolos)-1]
        self.simbolos = simbolos.split(',')  #Salva os simbolos do objeto automato

        tripla = quadrupla[len(simbolos)+3:] #passa para a variavel tripla a variavel quadrupla menos os estados

        #filtra os simbolos
        estados = re.search("\{.*?\}", tripla)
        estados = estados.group(0)
        estados = estados[1:len(estados)-1]
        self.estados = estados.split(',') #Salva os estados do objeto automato

        dupla = tripla[len(estados)+3:] #passa para a variavel dupla a variavel tripla menos os simbolos

        #filtra o estado inicial
        restante = dupla.split(',')
        self.inicial = restante[0]
        if(len(restante) == 2):
            self.finais = restante[1]
        else:        
            #filtra os estados finais
            finais = dupla[len(self.inicial)+1:]
            finais = finais[1:len(finais)-1]
            self.finais = finais.split(',')

### Testando para o exemplo dado pelo Lúcio e para o nosso automato da etapa 1

##### Automato Lucio

In [9]:
#abre o arquivo teste.txt e salva seu conteúdo na variável dados
openfile = open("teste.txt", "r")
dados = openfile.read()

teste = automato(dados)

print("estados:", teste.estados)
print("simbolos:", teste.simbolos)
print("Estado inicial:", teste.inicial)
print("Estados finais:", teste.finais)

estados: ['q0', 'q1', 'q2', 'q3']
simbolos: ['a', 'b']
Estado inicial: q0
Estados finais: ['q1', 'q3']


##### Automato Etapa 2

In [10]:
#abre o arquivo teste.txt e salva seu conteúdo na variável dados
openfile = open("automatop.txt", "r")
dados = openfile.read()

teste = automato(dados)

print("estados:", teste.estados)
print("simbolos:", teste.simbolos)
print("Estado inicial:", teste.inicial)
print("Estados finais:", teste.finais)

estados: ['q0', ' q1', ' q2', ' q3', ' q4', ' q5', ' q6', ' q7', ' q8', ' q9', ' q10', ' q11', ' q12', ' q13', ' q14', ' q15', ' q16', ' q17', ' q18', ' q19', ' q20', ' q21', ' q22', ' q23', ' q24']
simbolos: ['Liga', ' Desliga', ' Ficha Inserida', ' Alavanca', ' 7', ' Diamante', ' Cereja', ' Limão', ' Jackpot', ' Retira Premio', ' Libera Inserção de Ficha', ' Exibe Mensagem De Derrota']
Estado inicial: q0
Estados finais: q0
